In [1]:
import time
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime


In [2]:
import pymongo
#print(pymongo)
# Create the client
client = pymongo.MongoClient('localhost', 27017)

# Connect to our database
db = client['deCrime']

# Fetch our series collection
Scrapes = db['Scrapes']

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/nick/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [5]:
url = 'http://www.dpdconnect.com/dailybulletin.aspx'
browser.visit(url)

In [6]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mainTable = soup.find('div', id='gbox_tblDB')
listTable= soup.find('table', id="tblDB")
dateField = browser.find_by_id("mainContent_txtDate2")
dateSubmit = browser.find_by_id("mainContent_lbUpdate")
limitSelector = browser.find_by_css("#pager_center select")
typeSelector = browser.find_by_id("mainContent_ddlType2")
print(typeSelector.value)



AL


In [7]:
def nap(cat=.150):
    time.sleep(cat)


In [8]:
def rebaseHtml():
    nap()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    mainTable = soup.find('div', id='gbox_tblDB')
    listTable = soup.find('table', id="tblDB")
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    limitSelector = browser.find_by_css("#pager_center select")
    #limit_selector.select(5)
    limitSelector.select(10000)
    nap()
    limitSelector.click()
    typeSelector = browser.find_by_id("mainContent_ddlType2")
    typeSelector.select("AL")
    nap()


In [9]:
def changeDate(date):
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    dateNow=dateField.value
    print(f'{dateNow}=>{date}')
    dateField.fill(date)
    dateSubmit.click()
    rebaseHtml()

In [29]:
def scrapeAllIncidents():
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    mainTable = soup.find('div', id='gbox_tblDB')
    listTable = soup.find('table', id="tblDB")
    listIncidents = listTable.find_all('tr')
    print(len(listIncidents))
    print(listIncidents[0])
    for incident in listIncidents:
        caseType = incident['id'].split("<br>")[0].split("  ")[0]        
        caseNum = incident['id'].split("<br>")[0].split("  ")[1]
        tds = incident.find_all('td')
        #print(incident.find_all('td')[3]['title'])#Charge

        #print(incident.find_all('td')[1]['title'].split("(")[1].split(")")[0])#Race/Sex/Age tuple from longDe
        #print(incident.find_all('td')[4]['title'])#Last
        #print(incident.find_all('td')[5]['title'])#First
        #print(incident.find_all('td')[6]['title'])#Middle
        #print(incident.find_all('td')[7]['title'])#Race
        #print(incident.find_all('td')[8]['title'])#Sex
        #print(incident.find_all('td')[9]['title'])#Desc
        #print(incident.find_all('td')[10]['title'])#Address
        Scrapes.insert_one({"caseType": caseType,
                            "caseNum": caseNum,
                            "charge":tds[3]['title'],
                            #"rsa": tds[1]['title'].split("(")[1].split(")")[0],
                            "race": tds[7]['title'],
                            "sex": tds[8]['title'],
                            "description": tds[9]['title'],
                            "address": tds[10]['title']
                            })
        
    

In [22]:
changeDate('6/6/2015')

12/04/2012=>6/6/2015


In [23]:
changeDate('12/25/2015')
nap(.5)
scrapeAllIncidents()

6/6/2015=>12/25/2015
27
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  15014207&lt;br&gt;&lt;I&gt;1) Domestic Battery (m)...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  150142071) Domestic Battery (m)...">AR  15014207<br/><i>1) Domestic Battery (m)...</i></td><td role="gridcell" style="width: 432px;" title="Bond, Kiedren J (B /M/32) Arrest on chrg of 1) Domestic Battery (M) and 2) Contempt Of Court (F), at 525 W Macon St, Decatur, IL,  on 12/25/2015 09:40.">Bond, Kiedren J (B /M/32) Arrest on chrg of 1) Domestic Battery (M) and 2) Contempt Of Court (F), at 525 W Macon St, Decatur, IL,  on 12/25/2015 09:40.</td><td role="gridcell" style="width: 108px;" title="Sturdivant, L A">Sturdivant, L A</td><td role="gridcell" style="display:none;width: 150px;" title="1) Domestic Battery (M) and 2) Contempt Of Court (F),">1) Domestic Battery (M) and 2) Contempt Of Court (F),</td><td role="gridcell" style="display:none;width: 150px;" title="Bond">Bond</td><td 

In [33]:
#startDate = '10/09/2012' #experimentally determined enough for jazz
startDate = date(2015,7,27)
print (startDate.strftime("%m/%d/%Y"))

07/27/2015


In [25]:
def nextDate(dateOb):
    return dateOb + timedelta(days=1)

In [26]:
threeDaysLater = nextDate(nextDate(nextDate(startDate)))
print (threeDaysLater.strftime("%m/%d/%Y"))

12/07/2012


In [27]:
print(date.today().strftime("%m/%d/%Y"))
endDate= date(2012,10,16)
endDate = date.today()

03/29/2021


In [34]:
currentDate = startDate
while currentDate < endDate:
    nap()
    changeDate(currentDate.strftime("%m/%d/%Y"))
    nap()
    scrapeAllIncidents()
    currentDate=nextDate(currentDate)
    print("woohoo")

=>07/27/2015
68
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  15008161&lt;br&gt;&lt;I&gt;Contempt Of Court, F (f), &lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  15008161Contempt Of Court, F (f), ">AR  15008161<br/><i>Contempt Of Court, F (f), </i></td><td role="gridcell" style="width: 432px;" title="Chandler, Allen C (W /M/34) Arrest on chrg of Contempt Of Court, F (F), at 937 E Harrison Ave, Decatur, IL,  on 7/27/2015 01:25.">Chandler, Allen C (W /M/34) Arrest on chrg of Contempt Of Court, F (F), at 937 E Harrison Ave, Decatur, IL,  on 7/27/2015 01:25.</td><td role="gridcell" style="width: 108px;" title="Duffy, K A">Duffy, K A</td><td role="gridcell" style="display:none;width: 150px;" title="Contempt Of Court, F (F),">Contempt Of Court, F (F),</td><td role="gridcell" style="display:none;width: 150px;" title="Chandler">Chandler</td><td role="gridcell" style="display:none;width: 150px;" title="Allen">Allen</td><td role="gridcell" style="display:no

44
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  15004735&lt;br&gt;&lt;I&gt;Identity Theft, F (f), &lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  15004735Identity Theft, F (f), ">AR  15004735<br/><i>Identity Theft, F (f), </i></td><td role="gridcell" style="width: 432px;" title="Hanners, Shawn D (W /M/46) Arrest on chrg of Identity Theft, F (F), at 999 N Edward St/w Marietta St, Decatur, IL,  on 8/1/2015 15:59.">Hanners, Shawn D (W /M/46) Arrest on chrg of Identity Theft, F (F), at 999 N Edward St/w Marietta St, Decatur, IL,  on 8/1/2015 15:59.</td><td role="gridcell" style="width: 108px;" title="Appenzeller, J D">Appenzeller, J D</td><td role="gridcell" style="display:none;width: 150px;" title="Identity Theft, F (F),">Identity Theft, F (F),</td><td role="gridcell" style="display:none;width: 150px;" title="Hanners">Hanners</td><td role="gridcell" style="display:none;width: 150px;" title="Shawn">Shawn</td><td role="gridcell" style="display:none;wid

53
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  15006259&lt;br&gt;&lt;I&gt;Unlawful Visitation Int...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  15006259Unlawful Visitation Int...">AR  15006259<br/><i>Unlawful Visitation Int...</i></td><td role="gridcell" style="width: 432px;" title="Tuttle, Tonya M (W /F/32) Arrest on chrg of Unlawful Visitation Interference, M (M), at 707 W South Side Dr, Decatur, IL,  on 8/7/2015 00:01.">Tuttle, Tonya M (W /F/32) Arrest on chrg of Unlawful Visitation Interference, M (M), at 707 W South Side Dr, Decatur, IL,  on 8/7/2015 00:01.</td><td role="gridcell" style="width: 108px;" title="Heatherton, S A">Heatherton, S A</td><td role="gridcell" style="display:none;width: 150px;" title="Unlawful Visitation Interference, M (M),">Unlawful Visitation Interference, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Tuttle">Tuttle</td><td role="gridcell" style="display:none;width: 150px;" title="Tonya

54
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  15005610&lt;br&gt;&lt;I&gt;Domestic Battery, M (m), &lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  15005610Domestic Battery, M (m), ">AR  15005610<br/><i>Domestic Battery, M (m), </i></td><td role="gridcell" style="width: 432px;" title="Murphy, Devondre L (B /M/21) Arrest on chrg of Domestic Battery, M (M), at 800 Marietta, Decatur, IL,  on 8/12/2015 03:55.">Murphy, Devondre L (B /M/21) Arrest on chrg of Domestic Battery, M (M), at 800 Marietta, Decatur, IL,  on 8/12/2015 03:55.</td><td role="gridcell" style="width: 108px;" title="Perez, G S">Perez, G S</td><td role="gridcell" style="display:none;width: 150px;" title="Domestic Battery, M (M),">Domestic Battery, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Murphy">Murphy</td><td role="gridcell" style="display:none;width: 150px;" title="Devondre">Devondre</td><td role="gridcell" style="display:none;width: 150px;" title="L">

45
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  10011462&lt;br&gt;&lt;I&gt;1) Predatory Criminal S...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  100114621) Predatory Criminal S...">AR  10011462<br/><i>1) Predatory Criminal S...</i></td><td role="gridcell" style="width: 432px;" title="Wilhelm, Michael D (W /M/34) Arrest on chrg of 1) Predatory Criminal Sexual Assault (F), 2) Criminal Sexual Abuse (F), and 3) Obstructing Justice (F), at 1079 W Sunset Ave, Decatur, IL,  on 8/17/2015 17:10.">Wilhelm, Michael D (W /M/34) Arrest on chrg of 1) Predatory Criminal Sexual Assault (F), 2) Criminal Sexual Abuse (F), and 3) Obstructing Justice (F), at 1079 W Sunset Ave, Decatur, IL,  on 8/17/2015 17:10.</td><td role="gridcell" style="width: 108px;" title="Wicks, R">Wicks, R</td><td role="gridcell" style="display:none;width: 150px;" title="1) Predatory Criminal Sexual Assault (F), 2) Criminal Sexual Abuse (F), and 3) Obstructing Justice (F),">1) Predatory C

...

69
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  21001678&lt;br&gt;&lt;I&gt;Retail Theft, F (f), &lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  21001678Retail Theft, F (f), ">AR  21001678<br/><i>Retail Theft, F (f), </i></td><td role="gridcell" style="width: 432px;" title="Leach, Cody L (W /M/44) Arrest on chrg of Retail Theft, F (F), at 917 S 44th St, Decatur, IL,  on 3/3/2021 00:01.">Leach, Cody L (W /M/44) Arrest on chrg of Retail Theft, F (F), at 917 S 44th St, Decatur, IL,  on 3/3/2021 00:01.</td><td role="gridcell" style="width: 108px;" title="Hall, B A">Hall, B A</td><td role="gridcell" style="display:none;width: 150px;" title="Retail Theft, F (F),">Retail Theft, F (F),</td><td role="gridcell" style="display:none;width: 150px;" title="Leach">Leach</td><td role="gridcell" style="display:none;width: 150px;" title="Cody">Cody</td><td role="gridcell" style="display:none;width: 150px;" title="L">L</td><td role="gridcell" style="display:none;widt

41
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  20001702&lt;br&gt;&lt;I&gt;Possession Of Controlle...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  20001702Possession Of Controlle...">AR  20001702<br/><i>Possession Of Controlle...</i></td><td role="gridcell" style="width: 432px;" title="Fornwalt, Phillip Guy (W /M/52) Arrest on chrg of Possession Of Controlled Substance, F (F), at 799 N Moffet Ln/w Us Route 36, Decatur, IL,  on 3/8/2021 00:01.">Fornwalt, Phillip Guy (W /M/52) Arrest on chrg of Possession Of Controlled Substance, F (F), at 799 N Moffet Ln/w Us Route 36, Decatur, IL,  on 3/8/2021 00:01.</td><td role="gridcell" style="width: 108px;" title="Williams, J F">Williams, J F</td><td role="gridcell" style="display:none;width: 150px;" title="Possession Of Controlled Substance, F (F),">Possession Of Controlled Substance, F (F),</td><td role="gridcell" style="display:none;width: 150px;" title="Fornwalt">Fornwalt</td><td role="gridcell" style="

10
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  20012572&lt;br&gt;&lt;I&gt;Criminal Damage To Prop...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  20012572Criminal Damage To Prop...">AR  20012572<br/><i>Criminal Damage To Prop...</i></td><td role="gridcell" style="width: 432px;" title="Yarbary, Jericha A (B /F/32) Arrest on chrg of Criminal Damage To Property, M (M), at 1651 E Moore St, Decatur, IL,  on 3/14/2021 00:01.">Yarbary, Jericha A (B /F/32) Arrest on chrg of Criminal Damage To Property, M (M), at 1651 E Moore St, Decatur, IL,  on 3/14/2021 00:01.</td><td role="gridcell" style="width: 108px;" title="Throneburg, J T">Throneburg, J T</td><td role="gridcell" style="display:none;width: 150px;" title="Criminal Damage To Property, M (M),">Criminal Damage To Property, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Yarbary">Yarbary</td><td role="gridcell" style="display:none;width: 150px;" title="Jericha">Jericha</td><t

55
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  20007767&lt;br&gt;&lt;I&gt;City Ordinance /retail ...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  20007767City Ordinance /retail ...">AR  20007767<br/><i>City Ordinance /retail ...</i></td><td role="gridcell" style="width: 432px;" title="Harris-thomas, Aaliyah C (B /F/24) Arrest on chrg of City Ordinance /retail Theft, M (M), at 4224 N Prospect Dr, Decatur, IL,  on 3/19/2021 00:01.">Harris-thomas, Aaliyah C (B /F/24) Arrest on chrg of City Ordinance /retail Theft, M (M), at 4224 N Prospect Dr, Decatur, IL,  on 3/19/2021 00:01.</td><td role="gridcell" style="width: 108px;" title="Appenzeller, J D">Appenzeller, J D</td><td role="gridcell" style="display:none;width: 150px;" title="City Ordinance /retail Theft, M (M),">City Ordinance /retail Theft, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Harris-thomas">Harris-thomas</td><td role="gridcell" style="display:none;width: 15

32
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  21000812&lt;br&gt;&lt;I&gt;Domestic Battery, M (m), &lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  21000812Domestic Battery, M (m), ">AR  21000812<br/><i>Domestic Battery, M (m), </i></td><td role="gridcell" style="width: 432px;" title="Anderson, Vernon C (B /M/24) Arrest on chrg of Domestic Battery, M (M), at 1437 E Sedgwick St, Decatur, IL,  on 3/24/2021 16:46.">Anderson, Vernon C (B /M/24) Arrest on chrg of Domestic Battery, M (M), at 1437 E Sedgwick St, Decatur, IL,  on 3/24/2021 16:46.</td><td role="gridcell" style="width: 108px;" title="Bray, L R">Bray, L R</td><td role="gridcell" style="display:none;width: 150px;" title="Domestic Battery, M (M),">Domestic Battery, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Anderson">Anderson</td><td role="gridcell" style="display:none;width: 150px;" title="Vernon">Vernon</td><td role="gridcell" style="display:none;width: 150px;" 